# k-Nearest Neighbors
Finds k nearest data points to a given data point and outputs majority vote value of output classes in case of classification, and average value of target values in case of regression. KNN was first added in MADlib 1.10 with multiple updates in subsequent releases.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
# Greenplum Database 5.x on GCP (demo machine)
%sql postgresql://gpadmin@35.184.232.200:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [4]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.16-dev, git revision: rc/1.15.1-rc1-29-g0ba6155, cmake configuration time: Wed Feb 20 17:40:16 UTC 2019, build type: release, build system: Linux-2.6.32-754.6.3.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# 1.  Load data for classification

In [5]:
%%sql 
DROP TABLE IF EXISTS knn_train_data;

CREATE TABLE knn_train_data (
                    id integer, 
                    data integer[], 
                    label integer  -- Integer label means for classification
                    );

INSERT INTO knn_train_data VALUES
(1, '{1,1}', 1),
(2, '{2,2}', 1),
(3, '{3,3}', 1),
(4, '{4,4}', 1),
(5, '{4,5}', 1),
(6, '{20,50}', 0),
(7, '{10,31}', 0),
(8, '{81,13}', 0),
(9, '{1,111}', 0);

SELECT * FROM knn_train_data ORDER BY id;

Done.
Done.
9 rows affected.
9 rows affected.


id,data,label
1,"[1, 1]",1
2,"[2, 2]",1
3,"[3, 3]",1
4,"[4, 4]",1
5,"[4, 5]",1
6,"[20, 50]",0
7,"[10, 31]",0
8,"[81, 13]",0
9,"[1, 111]",0


# 2. Load data for regression

In [6]:
%%sql
DROP TABLE IF EXISTS knn_train_data_reg;

CREATE TABLE knn_train_data_reg (
                    id integer, 
                    data integer[], 
                    label float  -- Float label means for regression
                    );

INSERT INTO knn_train_data_reg VALUES
(1, '{1,1}', 1.0),
(2, '{2,2}', 1.0),
(3, '{3,3}', 1.0),
(4, '{4,4}', 1.0),
(5, '{4,5}', 1.0),
(6, '{20,50}', 0.0),
(7, '{10,31}', 0.0),
(8, '{81,13}', 0.0),
(9, '{1,111}', 0.0);

SELECT * FROM knn_train_data_reg ORDER BY id;

Done.
Done.
9 rows affected.
9 rows affected.


id,data,label
1,"[1, 1]",1.0
2,"[2, 2]",1.0
3,"[3, 3]",1.0
4,"[4, 4]",1.0
5,"[4, 5]",1.0
6,"[20, 50]",0.0
7,"[10, 31]",0.0
8,"[81, 13]",0.0
9,"[1, 111]",0.0


# 3. Load testing data

In [22]:
%%sql 
DROP TABLE IF EXISTS knn_test_data;

CREATE TABLE knn_test_data (
                    id integer, 
                    data integer[]
                    );

INSERT INTO knn_test_data VALUES
(1, '{2,1}'),
(2, '{2,6}'),
(3, '{15,40}'),
(4, '{12,1}'),
(5, '{2,90}'),
(6, '{50,45}');

SELECT * from knn_test_data ORDER BY id;

Done.
Done.
6 rows affected.
6 rows affected.


id,data
1,"[2, 1]"
2,"[2, 6]"
3,"[15, 40]"
4,"[12, 1]"
5,"[2, 90]"
6,"[50, 45]"


# 4. Run KNN for classification
Note that the nearest neighbors are sorted from closest to furthest from the corresponding test point.

In [15]:
%%sql
DROP TABLE IF EXISTS knn_result_classification;

SELECT * FROM madlib.knn(
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col name of id in train data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_classification',  -- Output table
                 3,                    -- Number of nearest neighbors
                 True,                 -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2' -- Distance function
                );

SELECT * from knn_result_classification ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction,k_nearest_neighbours
1,"[2, 1]",1.0,"[2, 1, 3]"
2,"[2, 6]",1.0,"[5, 4, 3]"
3,"[15, 40]",0.0,"[7, 6, 5]"
4,"[12, 1]",1.0,"[4, 5, 3]"
5,"[2, 90]",0.0,"[9, 6, 7]"
6,"[50, 45]",0.0,"[6, 7, 8]"


# 5. Run KNN for regression

In [16]:
%%sql
DROP TABLE IF EXISTS knn_result_regression;

SELECT * FROM madlib.knn(
                'knn_train_data_reg',  -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col Name of id in train data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_regression',  -- Output table
                 3,                    -- Number of nearest neighbors
                True,                  -- True to list nearest-neighbors by id
                'madlib.dist_norm2'    -- Distance function
                );

SELECT * FROM knn_result_regression ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction,k_nearest_neighbours
1,"[2, 1]",1.0,"[2, 1, 3]"
2,"[2, 6]",1.0,"[5, 4, 3]"
3,"[15, 40]",0.333333333333,"[7, 6, 5]"
4,"[12, 1]",1.0,"[4, 5, 3]"
5,"[2, 90]",0.0,"[9, 6, 7]"
6,"[50, 45]",0.0,"[6, 7, 8]"


# 6. List nearest neighbors only
(without doing classification or regression).  Note that the nearest neighbors are sorted from closest to furthest from the corresponding test point.

In [17]:
%%sql
DROP TABLE IF EXISTS knn_result_list_neighbors;

SELECT * FROM madlib.knn(
                'knn_train_data_reg',  -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col Name of id in train data
                NULL,                  -- NULL training labels means just list neighbors
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_list_neighbors', -- Output table
                3                      -- Number of nearest neighbors
                );

SELECT * FROM knn_result_list_neighbors ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,k_nearest_neighbours
1,"[2, 1]","[2, 1, 3]"
2,"[2, 6]","[5, 4, 3]"
3,"[15, 40]","[7, 6, 5]"
4,"[12, 1]","[4, 5, 3]"
5,"[2, 90]","[9, 6, 7]"
6,"[50, 45]","[6, 7, 8]"


# 7.  Weighted average
Run classification using weighted average

In [18]:
%%sql
DROP TABLE IF EXISTS knn_result_classification;

SELECT * FROM madlib.knn(
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col name of id in train data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_classification',  -- Output table
                 3,                    -- Number of nearest neighbors
                 True,                 -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2', -- Distance function
                 True                 -- For weighted average
                );

SELECT * FROM knn_result_classification ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction,k_nearest_neighbours
1,"[2, 1]",1,"[1, 2, 3]"
2,"[2, 6]",1,"[5, 4, 3]"
3,"[15, 40]",0,"[7, 6, 5]"
4,"[12, 1]",1,"[4, 5, 3]"
5,"[2, 90]",0,"[9, 6, 7]"
6,"[50, 45]",0,"[6, 7, 8]"


# 8. Use kd-tree algorithm 
Here we build a kd-tree to depth 4 and search half (8) of the 16 leaf nodes (i.e., 2^4)

In [19]:
%%sql
DROP TABLE IF EXISTS knn_result_classification_kd;

SELECT madlib.knn(
                'knn_train_data',        -- Table of training data
                'data',                  -- Col name of training data
                'id',                    -- Col name of id in train data
                NULL,                    -- Training labels
                'knn_test_data',         -- Table of test data
                'data',                  -- Col name of test data
                'id',                    -- Col name of id in test data
                'knn_result_classification_kd',  -- Output table
                 3,                      -- Number of nearest neighbors
                 True,                   -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2', -- Distance function
                 False,                  -- For weighted average
                 'kd_tree',              -- Use kd-tree
                 'depth=4, leaf_nodes=8' -- Kd-tree options
    
                 );
SELECT * FROM knn_result_classification_kd ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,k_nearest_neighbours
1,"[2, 1]","[1, 2, 3]"
2,"[2, 6]","[5, 4, 3]"
3,"[15, 40]","[7, 6, 5]"
4,"[12, 1]","[4, 5, 3]"
5,"[2, 90]","[9, 6, 7]"
6,"[50, 45]","[6, 7, 8]"


The result above is the same as brute force.  If we search just 1 leaf node, run-time will be faster but accuracy will be lower.  This shows up in this very small data set by not being able to find 3 nearest neighbors for all test points:

In [31]:
%%sql
DROP TABLE IF EXISTS knn_result_classification_kd;

SELECT madlib.knn(
                'knn_train_data',        -- Table of training data
                'data',                  -- Col name of training data
                'id',                    -- Col name of id in train data
                NULL,                    -- Training labels
                'knn_test_data',         -- Table of test data
                'data',                  -- Col name of test data
                'id',                    -- Col name of id in test data
                'knn_result_classification_kd',  -- Output table
                 3,                      -- Number of nearest neighbors
                 True,                   -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2', -- Distance function
                 False,                  -- For weighted average
                 'kd_tree',              -- Use kd-tree
                 'depth=4, leaf_nodes=1' -- Kd-tree options
    
                 );
SELECT * FROM knn_result_classification_kd ORDER BY id;

Done.
1 rows affected.
5 rows affected.


id,data,k_nearest_neighbours
1,"[2, 1]",[1]
2,"[2, 6]","[3, 2]"
3,"[15, 40]",[7]
5,"[2, 90]","[3, 2]"
6,"[50, 45]","[6, 8]"
